## Baixa dados sobre Covid da web

In [8]:
import requests

def download_file(id, destination):
    URL = "https://drive.google.com/uc?id=1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o&export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': id})
    token = get_confirm_token(response)
    if(token):
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params)
    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


if __name__ == "__main__":
    file_id = '1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o'
    destination = 'dados_covid.csv'
    download_file(file_id, destination)


## Cria banco de dados

In [9]:
import sqlite3
from sqlite3 import Error
import os

#https://www.sqlitetutorial.net/sqlite-python/


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def getPath():
    return os.path.join(os.getcwd(), 'dw_prova_grupo.db')

if __name__ == '__main__':
    print(getPath())
    conn = create_connection(getPath())

c:\Users\arthu\Desktop\2021.1\Xexeo\dw_trabalho_final_grupo\dw_prova_grupo.db


## Cria tabela estrela

In [10]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [11]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [12]:
def main():
    database = getPath()

    sql_create_vacinacao_covid_brasil = """ CREATE TABLE IF NOT EXISTS vacinacao_covid_brasil (
                                        FOREIGN KEY (paciente_fk_id) REFERENCES paciente(paciente_id),
                                        FOREIGN KEY(vacina_codigo_fk_id) REFERENCES vacina(vacina_codigo),
                                        FOREIGN KEY(estabelecimento_razao_social_fk_id) REFERENCES estabelecimento(estabelecimento_razao_social)
                                        data_importacao_rnds text,
                                        sistema_origem text,
                                        id_sistema_origem integer
                                    ); """

    sql_create_paciente_table = """ CREATE TABLE IF NOT EXISTS paciente (
                                        paciente_id text PRIMARY KEY,
                                        document_id text,
                                        paciente_idade integer,
                                        paciente_datanascimento text,
                                        paciente_enumsexobiologico char,
                                        paciente_racacor_valor text,
                                        paciente_racacor_codigo integer,
                                        paciente_endereco_coibgemunicipio integer,
                                        paciente_endereco_copais integer,
                                        paciente_endereco_nmmunicipio text,
                                        paciente_endereco_nmpais text,
                                        paciente_endereco_uf text,
                                        paciente_endereco_cep text,
                                        paciente_nacionalidade_enumnacionalidade char
                                    );"""

    sql_create_vacina_table = """ CREATE TABLE IF NOT EXISTS vacina (
                                        vacina_codigo integer PRIMARY KEY,
                                        vacina_nome text,
                                        vacina_numdose integer,
                                        vacina_descricao_dose text,
                                        vacina_dataaplicacao text,
                                        vacina_fabricante_referencia text,
                                        vacina_fabricante_nome text,
                                        vacina_lote text,
                                        vacina_categoria_nome text,
                                        vacina_categoria_codigo integer,
                                        vacina_grupoatendimento_nome text,
                                        vacina_grupoatendimento_codigo integer,
                                    );"""

    sql_create_estabelecimento_table = """ CREATE TABLE IF NOT EXISTS estabelecimento (
                                        estabelecimento_municipio_codigo integer PRIMARY KEY,
                                        estabelecimento_valor integer,
                                        estabelecimento_nofantasia text,
                                        estabelecimento_razaosocial text,
                                        estabelecimento_municipio_nome text,
                                        estabelecimento_uf text
                                    );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create historico table
        create_table(conn, sql_create_vacinacao_covid_brasil)
    else:
        print("Error! cannot create the database connection.")

In [13]:
if __name__ == '__main__':
    main()

near "FOREIGN": syntax error


## Insere dados na tabela

In [14]:
import pandas as pd

dados = pd.read_csv('dados_covid.csv')
dados.to_sql('users', conn, if_exists='append', index=False)


AttributeError: 'NoneType' object has no attribute 'cursor'